In [1]:
# Importing Necessary Libraries
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r"C:\Users\Aakash D\Downloads\Analytics\Projects\Python Mini Project (Vired)\Dataset Used\retail.csv")

In [3]:
df.shape

(1048575, 8)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   Invoice      1048575 non-null  object 
 1   StockCode    1048575 non-null  object 
 2   Description  1044203 non-null  object 
 3   Quantity     1048575 non-null  int64  
 4   InvoiceDate  1048575 non-null  object 
 5   Price        1048575 non-null  object 
 6   Customer ID  811893 non-null   float64
 7   Country      1048575 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 64.0+ MB


In [5]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country'],
      dtype='object')

In [6]:
# Checking for null
df.isna().sum()

Invoice             0
StockCode           0
Description      4372
Quantity            0
InvoiceDate         0
Price               0
Customer ID    236682
Country             0
dtype: int64

In [7]:
# Dropping the null values
df.dropna(inplace = True)

In [8]:
# Shape after dropping the null values
df.shape

(811893, 8)

In [9]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,1.12.2009 07:45,"6,95",13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,1.12.2009 07:45,"6,75",13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,1.12.2009 07:45,"2,1",13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,1.12.2009 07:45,"1,25",13085.0,United Kingdom


In [10]:
# Fixing the dtypes of attributes
df['Price'] = df['Price'].str.replace(",",".")
df['Price'] = df['Price'].astype("float64")
df['Customer ID'] = df['Customer ID'].astype("int64")

# Fixing the date
df['InvoiceDate']=pd.to_datetime(df.InvoiceDate)

### Finding out recency 

In [11]:
present_day = "01/01/2012"
present_day = pd.to_datetime(present_day)

recency = df.groupby(['Customer ID']).agg({'InvoiceDate': lambda x:((present_day-x.max()))})
recency

,InvoiceDate
Customer ID,
12346,347 days 13:43:00
12347,61 days 11:35:00
12348,97 days 10:47:00
12349,40 days 14:09:00
12350,332 days 07:59:00
...,...
18283,31 days 11:01:00
18284,569 days 11:29:00
18285,682 days 13:36:00


In [12]:
present_day-pd.to_datetime("1.12.2009 07:45")

Timedelta('1083 days 16:15:00')

 ### Finding out Frequency

In [13]:
#frequency = df.drop_duplicates(subset = "Invoice").groupby(["Customer ID"])[["Invoice"]].count()
frequency = df.groupby(["Customer ID"])[["Invoice"]].count()

In [14]:
frequency

,Invoice
Customer ID,
12346,48
12347,242
12348,51
12349,180
12350,17
...,...
18283,936
18284,29
18285,12


### Finding out the Monetary Value

In [15]:
# Finding the Total Revenue per customer
df['revenue'] = df['Price']*df['Quantity']
total_revenue = df.groupby(["Customer ID"])[["revenue"]].sum()
total_revenue

,revenue
Customer ID,
12346,-64.68
12347,5408.50
12348,2019.40
12349,4404.54
12350,334.40
...,...
18283,2528.65
18284,436.68
18285,427.00


In [16]:
# Finding the average Revenue per customer
average_revenue = df.groupby(["Customer ID"])[["revenue"]].mean()
average_revenue.rename(columns = {"revenue":"Avg_Rev"}, inplace = True)

### Merging all of it together

In [17]:
RFM = pd.concat([recency,frequency,total_revenue, average_revenue], axis = 1)

In [18]:
# Resetting the index to get the customer Id column
RFM.reset_index(inplace = True)

In [19]:
RFM.columns

Index(['Customer ID', 'InvoiceDate', 'Invoice', 'revenue', 'Avg_Rev'], dtype='object')

In [20]:
RFM.rename(columns = {"InvoiceDate":"Duration","Invoice":"Freq","revenue":"Tot_Rev"}, inplace = True)

In [21]:
# Final RFM Table
RFM

,Customer ID,Duration,Freq,Tot_Rev,Avg_Rev
0,12346,347 days 13:43:00,48,-64.68,-1.347500
1,12347,61 days 11:35:00,242,5408.50,22.349174
2,12348,97 days 10:47:00,51,2019.40,39.596078
3,12349,40 days 14:09:00,180,4404.54,24.469667
4,12350,332 days 07:59:00,17,334.40,19.670588
...,...,...,...,...,...
5919,18283,31 days 11:01:00,936,2528.65,2.701549
5920,18284,569 days 11:29:00,29,436.68,15.057931
5921,18285,682 days 13:36:00,12,427.00,35.583333
5922,18286,498 days 12:03:00,70,1188.43,16.977571


In [22]:
Task_1 = RFM.tail().sort_values('Customer ID', ascending = False).reset_index(drop = True)
Task_1

,Customer ID,Duration,Freq,Tot_Rev,Avg_Rev
0,18287,21 days 13:37:00,156,4177.89,26.781346
1,18286,498 days 12:03:00,70,1188.43,16.977571
2,18285,682 days 13:36:00,12,427.00,35.583333
3,18284,569 days 11:29:00,29,436.68,15.057931
4,18283,31 days 11:01:00,936,2528.65,2.701549


### End of Task 1

======================================================================================================

# Task 2

 ### To find out for which ranges of recency, is the company realizing around 75% of the total revenue.

In [23]:
Recency_Deciles = pd.qcut(x = RFM['Duration'], q = 10).reset_index()\
.rename(columns = {'Duration':'Recency_Deciles'})

In [24]:
Recency_Deciles.drop(columns = 'index', inplace = True)

In [25]:
Task_2 = pd.concat([RFM, Recency_Deciles], axis = 1)

In [26]:
Task_2 = Task_2.groupby('Recency_Deciles')[['Tot_Rev']].sum()

In [27]:
Task_2.reset_index(inplace = True)

In [28]:
Task_2

,Recency_Deciles,Tot_Rev
0,"(21 days 06:40:59.999999999, 32 days 12:04:36]",5711857.734
1,"(32 days 12:04:36, 43 days 13:59:12]",2968252.921
2,"(43 days 13:59:12, 61 days 10:34:24.000000001]",2275605.871
3,"(61 days 10:34:24.000000001, 87 days 18:32:36....",1533952.931
4,"(87 days 18:32:36.000000017, 135 days 08:26:00]",1262303.344
5,"(135 days 08:26:00, 216 days 09:30:36.000000004]",965468.294
6,"(216 days 09:30:36.000000004, 345 days 09:39:30]",586407.400
7,"(345 days 09:39:30, 436 days 12:19:24]",536943.362
8,"(436 days 12:19:24, 557 days 12:04:00]",398391.840
9,"(557 days 12:04:00, 1083 days 14:05:00]",164913.601


In [29]:
# For the Cumulative_Revenue
Task_2['CumSum_Rev'] = Task_2['Tot_Rev'].cumsum()

# For Total_revenue_across_Deciles
Task_2['Total_Rev_Across_Deciles'] = Task_2['Tot_Rev'].sum()

# For Percentage of Total_Revenue
Task_2['Perc_tot_Rev'] = (Task_2['CumSum_Rev']/Task_2['Total_Rev_Across_Deciles'])

In [30]:
# Final Task_2 Table
Task_2

,Recency_Deciles,Tot_Rev,CumSum_Rev,Total_Rev_Across_Deciles,Perc_tot_Rev
0,"(21 days 06:40:59.999999999, 32 days 12:04:36]",5711857.734,5.711858e+06,1.640410e+07,0.348197
1,"(32 days 12:04:36, 43 days 13:59:12]",2968252.921,8.680111e+06,1.640410e+07,0.529143
2,"(43 days 13:59:12, 61 days 10:34:24.000000001]",2275605.871,1.095572e+07,1.640410e+07,0.667865
3,"(61 days 10:34:24.000000001, 87 days 18:32:36....",1533952.931,1.248967e+07,1.640410e+07,0.761375
4,"(87 days 18:32:36.000000017, 135 days 08:26:00]",1262303.344,1.375197e+07,1.640410e+07,0.838325
5,"(135 days 08:26:00, 216 days 09:30:36.000000004]",965468.294,1.471744e+07,1.640410e+07,0.897181
6,"(216 days 09:30:36.000000004, 345 days 09:39:30]",586407.400,1.530385e+07,1.640410e+07,0.932928
7,"(345 days 09:39:30, 436 days 12:19:24]",536943.362,1.584079e+07,1.640410e+07,0.965661
8,"(436 days 12:19:24, 557 days 12:04:00]",398391.840,1.623918e+07,1.640410e+07,0.989947
9,"(557 days 12:04:00, 1083 days 14:05:00]",164913.601,1.640410e+07,1.640410e+07,1.000000


## End of Task 2

=======================================================================================================

# Task 3

To find out within the subset of data identified above, for what ranges of frequency is ~65% of the revenue in the subset realized.

In [31]:
Task_2

,Recency_Deciles,Tot_Rev,CumSum_Rev,Total_Rev_Across_Deciles,Perc_tot_Rev
0,"(21 days 06:40:59.999999999, 32 days 12:04:36]",5711857.734,5.711858e+06,1.640410e+07,0.348197
1,"(32 days 12:04:36, 43 days 13:59:12]",2968252.921,8.680111e+06,1.640410e+07,0.529143
2,"(43 days 13:59:12, 61 days 10:34:24.000000001]",2275605.871,1.095572e+07,1.640410e+07,0.667865
3,"(61 days 10:34:24.000000001, 87 days 18:32:36....",1533952.931,1.248967e+07,1.640410e+07,0.761375
4,"(87 days 18:32:36.000000017, 135 days 08:26:00]",1262303.344,1.375197e+07,1.640410e+07,0.838325
5,"(135 days 08:26:00, 216 days 09:30:36.000000004]",965468.294,1.471744e+07,1.640410e+07,0.897181
6,"(216 days 09:30:36.000000004, 345 days 09:39:30]",586407.400,1.530385e+07,1.640410e+07,0.932928
7,"(345 days 09:39:30, 436 days 12:19:24]",536943.362,1.584079e+07,1.640410e+07,0.965661
8,"(436 days 12:19:24, 557 days 12:04:00]",398391.840,1.623918e+07,1.640410e+07,0.989947
9,"(557 days 12:04:00, 1083 days 14:05:00]",164913.601,1.640410e+07,1.640410e+07,1.000000


In [32]:
# Using the qcut function for making Deciles
freq_dec = pd.qcut(x = RFM['Freq'], q = 10).reset_index()

# Droping the extra index columns
freq_dec.drop(columns = 'index', inplace = True)

# Renaming the column
freq_dec.rename(columns = {'Freq':'Freq_Deciles'}, inplace = True)

In [33]:
# Concatenating the Frequency Deciles variable with the RFM Table
RFM = pd.concat([RFM, freq_dec], axis = 1)
RFM

,Customer ID,Duration,Freq,Tot_Rev,Avg_Rev,Freq_Deciles
0,12346,347 days 13:43:00,48,-64.68,-1.347500,"(37.0, 53.0]"
1,12347,61 days 11:35:00,242,5408.50,22.349174,"(180.0, 319.7]"
2,12348,97 days 10:47:00,51,2019.40,39.596078,"(37.0, 53.0]"
3,12349,40 days 14:09:00,180,4404.54,24.469667,"(114.0, 180.0]"
4,12350,332 days 07:59:00,17,334.40,19.670588,"(16.0, 25.0]"
...,...,...,...,...,...,...
5919,18283,31 days 11:01:00,936,2528.65,2.701549,"(319.7, 12780.0]"
5920,18284,569 days 11:29:00,29,436.68,15.057931,"(25.0, 37.0]"
5921,18285,682 days 13:36:00,12,427.00,35.583333,"(8.0, 16.0]"
5922,18286,498 days 12:03:00,70,1188.43,16.977571,"(53.0, 76.0]"


In [34]:
# Calculating Total Revenue by Frequency Deciles
Task_3 = RFM.groupby('Freq_Deciles')[['Tot_Rev']].sum()
Task_3.reset_index(inplace = True)

# Calculating the Cumulative Sum of the Total_revenue
Task_3['CumSum'] = Task_3['Tot_Rev'].cumsum()

# Calculating the Total Revenue
Task_3['Total'] = Task_3['Tot_Rev'].sum()

# Calculating the Percentage Value 
Task_3['Perc_Total'] = Task_3['CumSum']/Task_3['Total']


In [35]:
# Final Revenue by Frequency Decile Table
Task_3

,Freq_Deciles,Tot_Rev,CumSum,Total,Perc_Total
0,"(0.999, 8.0]",72193.160,7.219316e+04,1.640410e+07,0.004401
1,"(8.0, 16.0]",242098.130,3.142913e+05,1.640410e+07,0.019159
2,"(16.0, 25.0]",274574.731,5.888660e+05,1.640410e+07,0.035897
3,"(25.0, 37.0]",383583.741,9.724498e+05,1.640410e+07,0.059281
4,"(37.0, 53.0]",603851.613,1.576301e+06,1.640410e+07,0.096092
5,"(53.0, 76.0]",773353.640,2.349655e+06,1.640410e+07,0.143236
6,"(76.0, 114.0]",1070299.192,3.419954e+06,1.640410e+07,0.208482
7,"(114.0, 180.0]",1630626.023,5.050580e+06,1.640410e+07,0.307885
8,"(180.0, 319.7]",2761936.172,7.812516e+06,1.640410e+07,0.476254
9,"(319.7, 12780.0]",8591580.896,1.640410e+07,1.640410e+07,1.000000


### End of Task 3

==========================================================================================================